In [1]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# First way to define the Hamiltonian
# two site Hubbard model

# system = [('C1', 'C2', 1)] is a list of tuples, where each tuple represents a bond
# between two atoms and the third element is the type of bond (singe or double).
# For now, we only support single bonds between carbon atoms. 
# For this type of bonds the default values of alpha and beta are -0.414 and -0.0533, respectively.
# In the future we are planning to support different types of bonds for different atoms.
system = [('C1', 'Cl2', 1.0), ('Cl2', 'F3', 2.0),
                    ('F3', 'Si4', 1.2), ('Si4', 'C1', 1.5)]
hubbard = HamHub(system,
                 alpha=-0.414, beta=-0.0533, u_onsite=np.array([1, 1]))

print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))

[('C', 'Cl', 1.0), ('Cl', 'F', 2.0), ('F', 'Si', 1.2), ('Si', 'C', 1.5)]
[[-0.41380839  0.          0.          0.        ]
 [ 0.         -0.47655051  0.          0.        ]
 [ 0.          0.         -0.64027609  0.        ]
 [ 0.          0.          0.         -0.29956945]]
[[-0.41380839 -0.75449848  0.         -0.56900877]
 [-0.75449848 -0.47655051 -0.82274152  0.        ]
 [ 0.         -0.82274152 -0.64027609 -0.56263344]
 [-0.56900877  0.         -0.56263344 -0.29956945]]
